## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте таблицу и изучите общую информацию о данных

Смотрим общую информацию о данных.

Импортируем библиотеки

In [51]:
from IPython.display import display
import pandas as pd
import altair as alt # https://altair-viz.github.io/index.html


import matplotlib.pyplot as plt

from pymystem3 import Mystem

Импортируем данные

In [52]:
data = pd.read_csv('datasets/data.csv')

Смотрим информацию о данных и содержимое

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [54]:
display(data.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Проанализируем информацию по столбцам. Начнем с количества детей. Заметим, что встречаются значения -1, что правдой быть не может. Таких значений крайне мало. Есть два варианта: можно предположить, что это неправильно введенное значение "1" (Например, пользователь под знаком "-" подразумевал тире, а результат сохранился как минус) и заменить его на 1, либо избавиться от этих данных (их крайне мало).
Заменим на значение 1 (вриведем все значения к неотрицатльным). Количество значений -1 даже на фоне значений 1 - пренебрежимо мало.

Возможен ещё вариант: -1 - у программистов часто используется, как знак отсутствия значения. В таком случае значение данного столбца, по логике, может быть равно нулю.

Напишем функцию для отображения уникальных значений столбца и общее количество значений по каждому уникальному значению.

In [55]:
def alt_value_counts(df:pd.DataFrame, col:str, title:str):
    # Функция для отображения уникальных значений столбца и общего количества значений по каждому уникальному значению
    # df - DataFrame. Данные для анализа
    # col - наименование колонки, информацию о которой необходимо отобразить.
    # title - Заголовок графика
    series_info = df.groupby(col)[col].count()
    data_info = pd.DataFrame({col:series_info.index, 'Количество':series_info.values})
    
    bars = alt.Chart(data_info, title = title).mark_bar().encode(
        x='Количество:Q',
        y=col + ":O"
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text='Количество:Q'
    )

    return (bars + text).properties(height = len(series_info) * 20)

In [59]:
alt_value_counts(data, 'children', 'Количество детей')

alt.LayerChart(...)

In [57]:
data['children'] = abs(data[['children']])

In [58]:
data = data[data['children'] != 20]

Рассмотрим колонку days_employed - общий трудовой стаж в днях. В этом столбце - большинство данных отрицательные. Скорее всего знак "-" взялся здесь случайно. Возьмем это значение по модулю. 


In [60]:
display(data.sort_values(by = 'days_employed', ascending = True).head(5))
display(data.sort_values(by = 'days_employed', ascending = False).head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба


Заметим, что есть как похожие на правду данные в днях, так и совсем нереальные (например - 401674.466633). При этом значения медианы и среднего арифметического - серьезно отличаются. Можно предположить - часть значений хранится не в днях, а в часах. Возможно, данные собраны из разных источников - в одном хранится в часах, в другом - в днях. 

Рассмотрим максимальные и минимальные значения в выборках больше и меньше вычисленного среднего арифметического.

Получили два совершенно разных диапазона значений. Попробуем найти Среднее арифметическое этих диапазонов, и во сколько раз они отличаются. Подтвердит или опровергнет это идею о том, что часть значений хранится в часах. 

Получили 365004 и 2353. Разница - в 155 раз. Есть сомнения, но возможно.. Есть ощущение, что к этим значениям, которые больше 300000 нечаянно прибавили 300000 (есть значения 400000+, поэтому тройку в начало не могли добавить). Сложно объяснить, как это могло произойти.. Но вот 65004 отличается от 2353 - в 27.6 раз, что очень близко к необходимым 24, которые доказывают, что данные хранятся в часах. 

Нужно отметить, что объем обеих выборок достаточно большой, чтобы там были возможны какие-то большие отклонения.

In [61]:
days_employed_median = data['days_employed'].median()
days_employed_mean = data['days_employed'].mean()

print('Медиана: {}'.format(days_employed_median))
print('Среднее арифметическое: {}'.format(days_employed_mean))

print('Максимальное в выборке "> ср.арифм.": {}'.format(data[data['days_employed'] > days_employed_mean]['days_employed'].max()))
print('Минимальное в выборке "> ср.арифм.": {}'.format(data[data['days_employed'] > days_employed_mean]['days_employed'].min()))
print('Максимальное в выборке "< ср.арифм.": {}'.format(data[data['days_employed'] < days_employed_mean]['days_employed'].max()))
print('Минимальное в выборке "< ср.арифм.": {}'.format(data[data['days_employed'] < days_employed_mean]['days_employed'].min()))

print('Среднее арифметическое "> ср.арифм.": {}'.format(data[data['days_employed'] > days_employed_mean]['days_employed'].mean()))
print('Среднее арифметическое "< ср.арифм.": {}'.format(data[data['days_employed'] < days_employed_mean]['days_employed'].mean()))

print('Объем выборки "> ср.арифм.": {}'.format(data[data['days_employed'] > days_employed_mean]['days_employed'].count()))
print('Объем выборки "< ср.арифм.": {}'.format(data[data['days_employed'] < days_employed_mean]['days_employed'].count()))




Медиана: -1204.1647137573468
Среднее арифметическое: 63141.23352703742
Максимальное в выборке "> ср.арифм.": 401755.40047533
Минимальное в выборке "> ср.арифм.": 328728.72060451825
Максимальное в выборке "< ср.арифм.": -24.14163324048118
Минимальное в выборке "< ср.арифм.": -18388.949900568383
Среднее арифметическое "> ср.арифм.": 365012.75258046377
Среднее арифметическое "< ср.арифм.": -2353.79881586843
Объем выборки "> ср.арифм.": 3438
Объем выборки "< ср.арифм.": 15846


Напишем соответствующую функцию, которая приведет значения к единому целому.

In [62]:
def days_employed_correct(row):
    try:
        value = row['days_employed']
        if value < days_employed_mean:
            return abs(row['days_employed'])
        if value >= days_employed_mean:
            return abs((row['days_employed'] - 300000) / 24)
        return value
    except:
        return 'Ошибка при обработке данных'
    
data['days_employed'] = data.apply(days_employed_correct, axis = 1)

Колонка dob_years - возраст клиента в годах. Посмотрим какие значения здесь встречаются. Заметим значение "0", которое явно некорректное. Поскольку таких значений очень мало на фоне выборки - решаем от них избавиться. Оставим только те, что больше нуля.

In [63]:
alt_counts_vis(data, 'dob_years')

NameError: name 'value_counts_vis' is not defined

In [168]:
data = data[data['dob_years'] > 0]

Колонка - education (образование). Замечаем раздвоение значений.

А также education_id - с ней, на удивление, всё нормально.

In [65]:
alt_value_counts(data, 'education', 'Образование')

alt.LayerChart(...)

In [66]:
alt_value_counts(data, 'education_id', 'Идентификатор образования')

alt.LayerChart(...)

Приведем все значения к единому регистру.

In [67]:
data['education'] = data['education'].str.lower()

family_status - семейное положение. Ничего необычного.

family_status_id - аналогично.

In [68]:
data['family_status'] = data['family_status'].str.lower()

In [69]:
alt_value_counts(data, 'family_status', 'Семейный статус')

alt.LayerChart(...)

In [70]:
alt_value_counts(data, 'family_status_id', 'Идентификатор семейного статуса')

alt.LayerChart(...)

gender. Замечаем всего одно неопределенное значение. Продолжаем без него.

In [72]:
alt_value_counts(data, 'gender', 'Пол')

alt.LayerChart(...)

In [73]:
data = data[data['gender'] != 'XNA']

income_type - тип занятости. Можно оставить всё как есть.

In [74]:
alt_value_counts(data, 'income_type', 'Тип дохода')

alt.LayerChart(...)

debt - имел ли задолженность по возврату кредитов. Всё корректно! 

In [75]:
alt_value_counts(data, 'debt', 'Задолженность')

alt.LayerChart(...)

purpose - цель получения кредита. Артефактов мы здесь не находим.

In [79]:
alt_value_counts(data, 'purpose', 'Цель получения кредита')

alt.LayerChart(...)

total_income. отсортируем по данной колонке и посмотрим начало и конец данных. Без учета значений NaN. Ничего необычного не замечаем.

In [80]:
display(data.sort_values(by ='total_income').head(10))
display(data.sort_values(by = 'total_income').tail(10))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14585,0,2467.460806,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667.263793,недвижимость
13006,0,2904.524546,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205.280566,заняться высшим образованием
16174,1,3642.820023,52,среднее,1,женат / замужем,0,M,сотрудник,0,21367.648356,приобретение автомобиля
1598,0,2488.587675,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695.101789,на проведение свадьбы
14276,0,1941.768908,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895.614355,недвижимость
10881,0,1973.188299,71,среднее,1,женат / замужем,0,M,пенсионер,0,22472.755205,операции со своей недвижимостью
18509,0,2936.596961,60,среднее,1,женат / замужем,0,F,пенсионер,0,23844.705592,покупка жилья для семьи
9070,0,1706.365238,60,среднее,1,не женат / не замужем,4,F,пенсионер,0,24457.666662,операции с коммерческой недвижимостью
10068,0,4959.893007,43,среднее,1,женат / замужем,0,F,сотрудник,0,25227.893684,высшее образование
12052,1,2273.483713,55,начальное,3,гражданский брак,1,F,пенсионер,0,25308.586849,дополнительное образование


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21415,0,NaN,54,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции с жильем
21423,0,NaN,63,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,сделка с автомобилем
21426,0,NaN,49,среднее,1,женат / замужем,0,F,сотрудник,1,NaN,недвижимость
21432,1,NaN,38,неоконченное высшее,2,не женат / не замужем,4,F,сотрудник,0,NaN,операции с жильем
21463,1,NaN,35,высшее,0,гражданский брак,1,M,сотрудник,0,NaN,на проведение свадьбы
21489,2,NaN,47,среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,высшее,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21448 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21448 non-null  int64  
 1   days_employed     19283 non-null  float64
 2   dob_years         21448 non-null  int64  
 3   education         21448 non-null  object 
 4   education_id      21448 non-null  int64  
 5   family_status     21448 non-null  object 
 6   family_status_id  21448 non-null  int64  
 7   gender            21448 non-null  object 
 8   income_type       21448 non-null  object 
 9   debt              21448 non-null  int64  
 10  total_income      19283 non-null  float64
 11  purpose           21448 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

Анализ данных начали с метода info(). С его помощью определили, что столбцы total_income и days_employed имеют пропущенные значения. Далее по очереди, с помощью метода value_counts(), проанализировали значения в каждом столбце. Почти в каждом столбце была какая-то своя проблема. 
Подытожим кратко все встрченные проблемы:

children - отрицательные значения (-1). Возможно данные были внесены некорректно пользователем - и на самом деле здесь 1. Возможно была ошибка в алгоритме сохранения данных. "-1" - как отсутствие значения (=0). Проблему можно было решить как и удалением неправильных строчек, так и их заменой на 0 или положительное. Был выбран вариант - привести к положительным значениям.

days_employed - по данному столбцу были обнаружены проблемы отсутствия данных и вывод значений в разном формате. Дни/часы.

dob_years - встречались значения равные 0, что не может быть правдой. Их было крайне мало, поэтому было решено строки с такими значениями удалить.

education - Одна и та же смысловая информация дублировалась из-за регистра. Было решено привести всё к одному регистру.

gender - замечено всего одно значение 'XNA'. Принято решение удалить строку с этим значением.

total_income - отсутствие значений.

В остальных колонках проблем обнаружено не было.

### Шаг 2. Предобработка данных

### Обработка пропусков

Вспомним, пропущенные значения у нас встречаются в колонках days_employed и total_income. Сравним их среднее арифметическое и медианы. Значения примерно одного порядка - будем заменять отсутствующие значения на соответствующую медиану.

Предварительно напишем функции нахождения мат.ожидания и медианы для Series.

In [81]:
def find_mean(serries):
    return serries.mean()

def find_median(serries):
    return serries.median()

In [82]:
print('Рабочий стаж. Среднее арифметическое: {}'.format(find_mean(data['days_employed'])))
print('Рабочий стаж. Медиана: {}'.format(find_median(data['days_employed'])))

print('Доход. Среднее арифметическое: {}'.format(find_mean(data['total_income'])))
print('Доход. Медиана: {}'.format(find_median(data['total_income'])))

Рабочий стаж. Среднее арифметическое: 2417.103886475152
Рабочий стаж. Медиана: 1902.310374457552
Доход. Среднее арифметическое: 167415.8994772097
Доход. Медиана: 145017.93753253992


In [83]:
def median_by_age_edu_proff(row):
    education = row['education']
    income_type = row['income_type']
    data_by_params = data[(data['education'] == education) & (data['income_type'] == income_type)]
    return data_by_params['total_income'].median()

In [84]:
def fillna_total_income(row):
    try:
        if row['total_income'] != row['total_income']:
            return median_by_age_edu_proff(row)
        return row['total_income']
    except:
        return 'Ошибка при обработке данных'

In [85]:
data['total_income'] = data.apply(fillna_total_income, axis = 1)

In [86]:
print(data['total_income'].value_counts())

136519.742601    819
114816.695032    342
159118.128873    298
165758.630174    236
201911.716329    187
                ... 
158475.831944      1
94866.149495       1
102584.459738      1
263541.296997      1
150014.128510      1
Name: total_income, Length: 19291, dtype: int64


In [87]:
data['days_employed'] = data['days_employed'].fillna(days_employed_median)

Проверим данные на отсутствующие значения.

In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21448 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21448 non-null  int64  
 1   days_employed     21448 non-null  float64
 2   dob_years         21448 non-null  int64  
 3   education         21448 non-null  object 
 4   education_id      21448 non-null  int64  
 5   family_status     21448 non-null  object 
 6   family_status_id  21448 non-null  int64  
 7   gender            21448 non-null  object 
 8   income_type       21448 non-null  object 
 9   debt              21448 non-null  int64  
 10  total_income      21448 non-null  float64
 11  purpose           21448 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод



Пропущенные значения у нас в двух колонках: days_employed и income_type. Их количество - около двух тысяч, что составляет существенную часть от общей выборки (около 10%). По этой причине удалить эти строки мы не можем - удаление этих данных может повлиять на итоговые результаты анализа. Нужно менять отсутствующие значения на что-то подходящее по смыслу. Варианта два - менять на среднее арифметичское или медиану. По сути, отличаются они не критически (исходя из следующих задач проекта). Было решено заменить пропущенные значения на соответсвующие медианы.

### Замена типа данных

Просмотрим какие типы данных в нашем датасете сейчас.

In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21448 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21448 non-null  int64  
 1   days_employed     21448 non-null  float64
 2   dob_years         21448 non-null  int64  
 3   education         21448 non-null  object 
 4   education_id      21448 non-null  int64  
 5   family_status     21448 non-null  object 
 6   family_status_id  21448 non-null  int64  
 7   gender            21448 non-null  object 
 8   income_type       21448 non-null  object 
 9   debt              21448 non-null  int64  
 10  total_income      21448 non-null  float64
 11  purpose           21448 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Приведем столбцы days_employed и total_income к типу int с помощью метода astype().

In [90]:
data['days_employed'] = data['days_employed'].round()
data['total_income'] = data['total_income'].round()

In [91]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

Проверяем:

In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21448 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21448 non-null  int64 
 1   days_employed     21448 non-null  int32 
 2   dob_years         21448 non-null  int64 
 3   education         21448 non-null  object
 4   education_id      21448 non-null  int64 
 5   family_status     21448 non-null  object
 6   family_status_id  21448 non-null  int64 
 7   gender            21448 non-null  object
 8   income_type       21448 non-null  object
 9   debt              21448 non-null  int64 
 10  total_income      21448 non-null  int32 
 11  purpose           21448 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Стаж, измеряемый в количестве дней, - не может быть дробным. Приведем его к целочисленному значению с помощью метода astype().
total_income - доход - у нас тоже дробное. Но, определенно, цифры после запятой никакого влияния на результаты не окажут. С целью более компактного отображения данных можно тоже привести к int.

### Обработка дубликатов

Посмотрим сколько всего в данных дубликатов.

In [93]:
print('Количество дубликатов: {}'.format(data.duplicated().sum()))

Количество дубликатов: 71


Удаляем дубликаты и обновляем нумерацию с помощью функций drop_duplicates() и reset_index(). *drop = true*

In [94]:
data = data.drop_duplicates().reset_index(drop = True)

Проверяем:

In [95]:
print('Количество дубликатов: {}'.format(data.duplicated().sum()))

Количество дубликатов: 0


### Вывод

Был обнаружен 71 дубликат. Все они были удалены, индексы обновлены.
Причиной появления может быть ошибочно-повторное внесение информации в базу данных.

### Лемматизация

Выведем список всех целей, которые встречаются в выборке.

In [96]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [97]:
purposes = data['purpose'].unique()

In [98]:
purposes_data = pd.DataFrame(purposes)

In [99]:
purposes_data.columns = ['purpose']

 Заметим, что некоторые значения несут один и тот же смысл, но при этом значения отличаются. Например 'образование' и 'дополнительное образование'. Всё это можно объединить под целью "образование". Для этого каждое значение purpose лемматизируем, а затем снова склеим.

In [100]:
def lemma_func(row):
    m = Mystem() 
    result = ''.join(m.lemmatize(row['purpose']))
    return result.rstrip('\n')

In [101]:
%%time
purposes_data['purpose_lemma'] = purposes_data.apply(lemma_func, axis = 1)

Wall time: 44.5 s


In [102]:
data = data.merge(purposes_data, left_on='purpose', right_on='purpose')

In [103]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,покупка жилье
1,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,покупка жилье
2,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255764,покупка жилья,покупка жилье
3,0,1549,48,среднее,1,женат / замужем,0,F,компаньон,0,157246,покупка жилья,покупка жилье
4,0,414,41,среднее,1,женат / замужем,0,M,госслужащий,0,118552,покупка жилья,покупка жилье


In [104]:
#%%time
#data['purpose_lemma'] = data.head(3).apply(lemma_func, axis = 1)

В полученном столбце будем искать слова, которые указывают на определенную группу целей. 
'жилье' или 'недвижимость' - 'недвижимость'
'автомобиль' - 'автомобиль'
'свадьба' - 'свадьба'
'образование' - 'образование'

Отдельно выделим 'прочее' на случай, если что-то не учли.

In [105]:
def PurposeType_func(row):
    try:
        query = row['purpose_lemma']
        if 'жилье' in query or 'недвижимость' in query:
            return 'недвижимость'
        if 'автомобиль' in query:
            return 'автомобиль'
        if 'свадьба' in query:
            return 'свадьба'
        if 'образование' in query:
            return 'образование'
        return 'прочее'
    except:
        return 'ошибка при обработке данных'

In [106]:
data['purpose_type'] = data.apply(PurposeType_func, axis = 1)

Проверяем:

In [107]:
alt_value_counts(data, 'purpose_type', 'Тип цели получения кредита')

alt.LayerChart(...)

### Вывод

Изначально у нас было множество целей, многие из которых означали одно и то же, но именовались по-разному. Необходимо было объединить эти значения в единые группы. С помощью лемматизации мы привели все слова в строке 'purpose' к словарной форме. А затем склеили обратно. После чего с помощью поиска в этих строках по ключевым словам удалось цели разбить на следующие группы:
недвижимость
автомобиль
образование
свадьба 

### Категоризация данных

In [108]:
def age_type(row):
    try:
        age = row['dob_years']
        if age < 25:
            return 'до 25'
        if age >=25 and age < 35:
            return '25-34'
        if age >=35 and age < 45:
            return '35-44'
        if age >=45 and age < 65:
            return '45-64'
        if age >=65:
            return '65 и выше'
        return 'прочие'
    except:
        return 'ошибка'

In [109]:
data['age_type'] = data.apply(age_type, axis = 1)

In [110]:
alt_value_counts(data, 'age_type', 'Возрастные категории')

alt.LayerChart(...)

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Приведем числовые значения debt к "понятным" строковым.

In [111]:
def debttype_func(row):
    try:
        if row['debt'] == 1:
            return 'Возвращено не вовремя'
        return 'Возвращено вовремя'
    except:
        return 'ошибка при обработке данных'

In [112]:
data['debt_type'] = data.apply(debttype_func, axis = 1)

Посмотрим по сколько раз встречаются заёмщики с тем или иным количеством детей.

In [113]:
alt_value_counts(data, 'children', 'Количество детей')

alt.LayerChart(...)

Разобьем все наблюдения по группам: если >0 детей - "есть дети", иначе - "нет детей". Подсчитаем количество должников и их процент от общего количества.

In [114]:
def haschildren_func(row):
    try:
        if row['children'] > 0:
            return 'Есть дети'
        return 'Нет детей'
    except:
        return 'ошибка при обработке данных'

In [115]:
data['has_children'] = data.apply(haschildren_func, axis = 1)

dep_children_debt = data.pivot_table(
    index = 'has_children', 
    columns = 'debt_type', 
    values = 'debt', 
    aggfunc = ['count']
)

dep_children_debt['Процент несвоевременных возвратов от общего количества'] = dep_children_debt['count']['Возвращено не вовремя'] * 100 / (dep_children_debt['count']['Возвращено вовремя'] + dep_children_debt['count']['Возвращено не вовремя'])

display(dep_children_debt)

count                        \
debt_type    Возвращено вовремя Возвращено не вовремя   
has_children                                            
Есть дети                  6617                   670   
Нет детей                 13027                  1063   

             Процент несвоевременных возвратов от общего количества  
debt_type                                                            
has_children                                                         
Есть дети                                              9.194456      
Нет детей                                              7.544358

In [116]:
dep_children_amount_debt = data.pivot_table(
    index = 'children', 
    columns = 'debt_type', 
    values = 'debt', 
    aggfunc = ['count']
)

dep_children_amount_debt['Процент несвоевременных возвратов от общего количества'] = dep_children_amount_debt['count']['Возвращено не вовремя'] * 100 / (dep_children_amount_debt['count']['Возвращено вовремя'] + dep_children_amount_debt['count']['Возвращено не вовремя'])

display(dep_children_amount_debt.fillna(0))

count                        \
debt_type Возвращено вовремя Возвращено не вовремя   
children                                             
0                    13027.0                1063.0   
1                     4410.0                 445.0   
2                     1858.0                 194.0   
3                      303.0                  27.0   
4                       37.0                   4.0   
5                        9.0                   0.0   

          Процент несвоевременных возвратов от общего количества  
debt_type                                                         
children                                                          
0                                                   7.544358      
1                                                   9.165808      
2                                                   9.454191      
3                                                   8.181818      
4                                                   9.756098      
5                                                   0.000000

### Вывод

Получили, следующие цифры 9,2% заемщиков с детьми не возвращают в срок, а без детей - всего 7,5.
Это значит - зависимость между наличием детей и возвратом кредита в срок есть.
Объяснить можно тем, что часть доходов человека уходит на детей. А также - гораздо сложнее планировать бюджет при наличии детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [117]:
dep_family_status_debt = data.pivot_table(
    index = 'family_status', 
    columns = 'debt_type', 
    values = 'debt', 
    aggfunc = ['count']
)

dep_family_status_debt['Процент несвоевременных возвратов от общего количества'] = dep_family_status_debt['count']['Возвращено не вовремя'] * 100 / (dep_family_status_debt['count']['Возвращено вовремя'] + dep_family_status_debt['count']['Возвращено не вовремя'])

display(dep_family_status_debt)

count                        \
debt_type             Возвращено вовремя Возвращено не вовремя   
family_status                                                    
в разводе                           1109                    84   
вдовец / вдова                       892                    63   
гражданский брак                    3753                   385   
женат / замужем                    11362                   928   
не женат / не замужем               2528                   273   

                      Процент несвоевременных возвратов от общего количества  
debt_type                                                                     
family_status                                                                 
в разводе                                                       7.041073      
вдовец / вдова                                                  6.596859      
гражданский брак                                                9.304012      
женат / замужем                                                 7.550854      
не женат / не замужем                                           9.746519

### Вывод

По цифрам можно сделать вывод, что семейное положение влияет на возврат в срок. А вот обосновать сложнее. Графы 'в разводе' и 'вдовец / вдова' отличаются небольшим количеством наблюдений и, следовательно, большим удельным весом каждого события (очень большая погрешность).
Если вынести эти статусы за скобки - можно увидеть следующую закономерность. Отсутствие партнера в отношениях - повышает риски невозврата.
Не женат / не замужем - 9,8%
гражданский брак - 9.3%
женат / замужем - 7.5%
Наличие мужа/жены значительно снижает вероятность невозврата, поскольку оба партнера полностью разделяют все риски.
В гражданском же браке это явление имеет выборочный характер, т.к. юридически партнеры, все-таки, друг от друга не зависят.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Было решено разделить доход на три уровня - низкий, средний, высокий. Диапазоны подобраны таким образом, чтобы на уровень среднего дохода приходилось около 50% наблюдений, а низкий и высокий - около 25%.

In [118]:
def incomelevel_func(row):
    try:
        if row['total_income'] < 110000:
            return 'низкий'
        if row['total_income'] > 200000:
            return 'высокий'
        return 'средний'
    except:
        return 'ошибка при обработке данных'

In [119]:
data['income_level'] = data.apply(incomelevel_func, axis = 1)

dep_income_level_debt = data.pivot_table(index = 'income_level', columns = 'debt_type', values = 'debt', aggfunc = ['count'])
dep_income_level_debt['Процент несвоевременных возвратов от общего количества'] = dep_income_level_debt['count']['Возвращено не вовремя'] * 100 / (dep_income_level_debt['count']['Возвращено вовремя'] + dep_income_level_debt['count']['Возвращено не вовремя'])

display(dep_income_level_debt)

count                        \
debt_type    Возвращено вовремя Возвращено не вовремя   
income_level                                            
высокий                    4866                   364   
низкий                     5178                   455   
средний                    9600                   914   

             Процент несвоевременных возвратов от общего количества  
debt_type                                                            
income_level                                                         
высокий                                                6.959847      
низкий                                                 8.077401      
средний                                                8.693171

### Вывод

Как видим по цифрам - прямой зависимости нет. Обосновать расхождения можно тем, что основными заемщиками, как правило, являются люди со средним уровнем дохода. Людям с высоким уровнем дохода выплачивать вовремя объективно проще. Людям же с низким уровнем дохода банки гораздо осторожнее дают кредиты и часть потенциальных должников отсеивается ещё на этапе одобрения кредита. Очевидно, уровень дохода является ключевым показателем при одобрении кредита. С этим и связана относительная "ровность" результатов и объяснение небольших отклонений. 

- Как разные цели кредита влияют на его возврат в срок?

In [120]:
dep_purpose_debt = data.pivot_table(
    index = 'purpose_type', 
    columns = 'debt_type', 
    values = 'debt', 
    aggfunc = ['count']
)

dep_purpose_debt['Процент несвоевременных возвратов от общего количества'] = dep_purpose_debt['count']['Возвращено не вовремя'] * 100 / (dep_purpose_debt['count']['Возвращено вовремя'] + dep_purpose_debt['count']['Возвращено не вовремя'])

display(dep_purpose_debt)

count                        \
debt_type    Возвращено вовремя Возвращено не вовремя   
purpose_type                                            
автомобиль                 3889                   401   
недвижимость               9994                   780   
образование                3629                   369   
свадьба                    2132                   183   

             Процент несвоевременных возвратов от общего количества  
debt_type                                                            
purpose_type                                                         
автомобиль                                             9.347319      
недвижимость                                           7.239651      
образование                                            9.229615      
свадьба                                                7.904968

### Вывод

Зависимость цели кредита также зависит от возврат в срок.
Процент несвоевременных возвратов по группам "недвижимость" и "свадьба" примерно одинаков. А вот группы "автомобиль" и "образование" характеризуются дополнительными рисками. 
Появление автомобиля, как правило, приводит к дополнительным расходам (в т.ч., порой незапланированным). Это увеличивает риски.
Период получения образования же характеризуется снижением времени, которое человек может посвящать заработку. Совмещать образование и работу сложнее - следовательно возрастают риски снижения дохода - следовательно повышаются риски несвоевременного возврата. 

### Шаг 4. Общий вывод

Мы провели анализ статистики о платёжеспособности клиентов.
Была проведена предобработка данных, в ходе которой были устранены дубликаты, заполнены пропуски. Было произведено приведение к необходимым типам данных, выделены леммы, а также категоризированы данные. 
После чего был произведен анализ зависимостей возврата кредита в срок от количества детей/семейного положения/цели взятия кредита/уровня дохода. В той или иной степени вероятность возврата кредита зависит от всех этих критериев, за исключением, пожалуй, уровня дохода. Возможно, по причине того, что этот показатель - ключевой при нынешней оценке платежеспособности.

Для итоговой системы можно подобрать для каждого типа свой эталон и свои весовые коэффициенты, которые будут принимать участие в расчете платежеспособности заемщиков.

Например, наличие детей: детей нет - весовой коэффициент - 1,0
                         дети есть - весовой коэффициент - 1,0 ( исходя из наших расчетов: 1 + (9,2-7,5)/7,5 = 1,2)

Чем больше кэффициент - тем больше риски.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.